# install prerequisites

### WARNING: 
some packages will probably be upgraded some packages.
Create a virtual env if you want to keep your packages intact

In [ ]:
!pip3 install --user pillow==11.1.0 numba==0.61.0
!pip3 install --user numpy==2.1.3 scipy==1.15.2
!pip3 install --user imageio==2.37.0

## Load an image (1000 x 1000 pixels) and display it

In [ ]:
import numpy
from imageio import imread

mat = imread('sample.jpg').astype('f8')
m, n = mat.shape


In [ ]:
%matplotlib inline
from matplotlib import pyplot
pyplot.imshow(mat, cmap='gray')
pyplot.colorbar()
pyplot.title('Grayscale Image')
pyplot.show()

# Pure Python Implementation

### utility function that generates a matrix as a list of lists

In [ ]:
def generate_zeros_matrix(m, n):
    return [[0 for _ in range(n)] for _ in range(m)]

### function that apply the filter by iterating over columns

In [ ]:
def laplacian_filter_pure_python(mat, retval):
    for c in range(1, n - 1):
        for r in range(1, m - 1):
            retval[r][c] = -4.0*mat[r][c] +     \
                                mat[r-1][c] +   \
                                mat[r+1][c] +   \
                                mat[r][c - 1] + \
                                mat[r][c + 1]

### function that apply the filter by iterating over rows

In [ ]:
def laplacian_filter_pure_python_row_sweep(mat, retval):
    for r in range(1, m - 1):
        for c in range(1, n - 1):
            retval[r][c] = -4.0*mat[r][c] +      \
                                mat[r-1][c] +    \
                                mat[r+1][c] +    \
                                mat[r][c - 1] +  \
                                mat[r][c + 1]

In [ ]:
# convert the numpy image to a list of lists
mat_list_of_lists = [[pixel for pixel in row] for row in mat]

In [ ]:
# create the matrix that will hold the "filtered" image (with the edges)
mat_edges = generate_zeros_matrix(m, n)

In [ ]:
# apply the filter and display the convolved image
laplacian_filter_pure_python(mat_list_of_lists, mat_edges)

# convert the list of lists back to a numpy array
mat_edges = numpy.array(mat_edges)

# Display the convolved image with a narrowed colorbar
pyplot.imshow(mat_edges > 0, cmap='gray')
pyplot.colorbar()
pyplot.title('Laplacian Filter - Edges Detected')
pyplot.tight_layout()
pyplot.show()

In [ ]:
timeit_pure_python = %timeit -o laplacian_filter_pure_python(mat_list_of_lists, mat_edges)
time_pure_python = timeit_pure_python.best

# save the value into a dictionary
time_results = {'Pure Python': time_pure_python}

In [ ]:
timeit_pure_python_row_sweep = %timeit -o laplacian_filter_pure_python_row_sweep(mat_list_of_lists, mat_edges)
time_row_sweep = timeit_pure_python_row_sweep.best

# save the value into a dictionary
time_results['Pure Python Row Sweep'] = time_row_sweep

# plot the results
pyplot.bar(time_results.keys(), time_results.values())
pyplot.ylabel('Time (s)')
pyplot.title('Execution Time')
pyplot.show()

# Numpy

## Iterate over the numpy array with python loops

In [ ]:
def laplacian_filter_numpy_python_loops(mat, retval):
    m, n = mat.shape
    for r in range(1, m - 1):
        for c in range(1, n - 1):
            retval[r][c] = -4.0*mat[r][c] +     \
                                mat[r-1][c] +   \
                                mat[r+1][c] +   \
                                mat[r][c - 1] + \
                                mat[r][c + 1]
    return retval

In [ ]:
mat_edges = numpy.zeros_like(mat)
laplacian_filter_numpy_python_loops(mat, mat_edges)
pyplot.imshow(mat_edges > 0, cmap='gray')
pyplot.colorbar()
pyplot.title('Laplacian Filter - Edges Detected')
pyplot.tight_layout()
pyplot.show()

In [ ]:
timeit_numpy_python_loop = %timeit -o laplacian_filter_numpy_python_loops(mat, mat_edges)
time_numpy_python_loop = timeit_numpy_python_loop.best

# save the value into a dictionary
time_results['Numpy Python Loops'] = time_numpy_python_loop

# plot the results
pyplot.bar(time_results.keys(), time_results.values())
pyplot.ylabel('Time (s)')
pyplot.title('Execution Time')
pyplot.xticks(rotation=45)
pyplot.tight_layout()
pyplot.show()

## Numpy vectorized operations only (no explicit python loops)

In [ ]:
from functools import reduce
def laplacian_filter_numpy_roll(mat):
    mat_ext = numpy.pad(mat, 1, 'constant')

    rolled = [
        numpy.roll(mat_ext, 1, 0),
        numpy.roll(mat_ext, -1, 0),
        numpy.roll(mat_ext, 1, 1),
        numpy.roll(mat_ext, -1, 1)
    ]
    retval = reduce(numpy.add, rolled)[1:-1, 1:-1] - 4.0*mat
    return retval

In [ ]:
mat_edges = laplacian_filter_numpy_roll(mat)
pyplot.imshow(mat_edges > 0, cmap='gray')
pyplot.colorbar()
pyplot.title('Laplacian Filter - Edges Detected')
pyplot.tight_layout()
pyplot.show()

In [ ]:
timeit_numpy_roll = %timeit -o laplacian_filter_numpy_roll(mat)
time_numpy_roll = timeit_numpy_roll.best

# save the value into a dictionary
time_results['Numpy Roll'] = time_numpy_roll

# plot the results in log scale
pyplot.bar(time_results.keys(), time_results.values())
pyplot.ylabel('Time (s)')
pyplot.title('Execution Time')
pyplot.xticks(rotation=45)
pyplot.tight_layout()
pyplot.yscale('log')  # set y-axis to log scale
pyplot.show()

## Numpy operations only (lower memory footprint)

In [ ]:
def laplacian_filter_numpy_map_reduce(mat):

    mat_ext = numpy.pad(mat, 1, 'constant')

    retval = reduce(
        numpy.add,
        map(lambda args: numpy.roll(
            mat_ext, shift=args[0],
            axis=args[1]),
            [[-1, 0], [1, 0], [-1, 1], [1, 1]])
    )

    return retval[1:-1, 1:-1] - 4.0*mat

In [ ]:
mat_edges = laplacian_filter_numpy_map_reduce(mat)
pyplot.imshow(mat_edges > 0, cmap='gray')
pyplot.colorbar()
pyplot.title('Laplacian Filter - Edges Detected')
pyplot.tight_layout()
pyplot.show()

In [ ]:
timeit_numpy_map_reduce =  %timeit -o laplacian_filter_numpy_map_reduce(mat)
time_numpy_map_reduce = timeit_numpy_map_reduce.best

# save the value into a dictionary
time_results['Numpy Map-Reduce'] = time_numpy_map_reduce

# plot the results in log scale
pyplot.bar(time_results.keys(), time_results.values())
pyplot.ylabel('Time (s)')
pyplot.title('Execution Time')
pyplot.xticks(rotation=45)
pyplot.tight_layout()
pyplot.yscale('log')  # set y-axis to log scale
pyplot.show()

## Pure python code + numba

https://github.com/numba/numba

In [ ]:
from numba import jit

### only difference between "laplacian_filter_numpy_python_loops" is "@jit"

In [ ]:
@jit
def laplacian_filter_numba(mat, retval):
    m, n = mat.shape
    for c in range(1, n - 1):
        for r in range(1, m - 1):
            retval[r, c] = -4.0*mat[r, c] +     \
                                mat[r-1, c] +   \
                                mat[r+1, c] +   \
                                mat[r, c - 1] + \
                                mat[r, c + 1]

### iterate over the rows instead of columns 

In [ ]:
@jit
def laplacian_filter_numba_row(mat, retval):
    m, n = mat.shape
    for r in range(1, m - 1):
        for c in range(1, n - 1):
            retval[r, c] = -4.0*mat[r, c] +     \
                                mat[r-1, c] +   \
                                mat[r+1, c] +   \
                                mat[r, c - 1] + \
                                mat[r, c + 1]

In [ ]:
laplacian_filter_numba(mat, mat_edges)
pyplot.imshow(mat_edges > 0, cmap='gray')
pyplot.colorbar()
pyplot.title('Laplacian Filter - Edges Detected')
pyplot.tight_layout()
pyplot.show()

In [ ]:
timeit_numba = %timeit -o laplacian_filter_numba(mat, mat_edges)
time_numba = timeit_numba.best

# save the value into a dictionary
time_results['Numba'] = time_numba


In [ ]:
timeit_numpy_python_loop = %timeit -o laplacian_filter_numba_row(mat, mat_edges)
time_numba_python_loop = timeit_numpy_python_loop.best

time_results['Numba Python Loop'] = time_numba_python_loop

pyplot.bar(time_results.keys(), time_results.values())
pyplot.ylabel('Time (s)')
pyplot.title('Execution Time')
pyplot.xticks(rotation=45)
pyplot.tight_layout()
pyplot.yscale('log')  # set y-axis to log scale
pyplot.show()

# Cython

https://github.com/cython/cython

In [ ]:
!cat cython_bindings/*.pyx

In [ ]:
!cat cython_bindings/setup.py

In [ ]:
!rm -fvr cython_bindings/*.so
!cd cython_bindings && python3 setup.py build_ext --inplace

In [ ]:
from cython_bindings.cython_binding import laplacian_filter_py

### prepare the data (flatten it)

In [ ]:
m, n = mat.shape
mat_flat = mat.flatten()
mat_edges_flat = numpy.zeros_like(mat_flat)

In [ ]:
laplacian_filter_py(mat_flat, m, n, mat_edges_flat)
pyplot.imshow(mat_edges.reshape(m, n) > 0, cmap='gray')
pyplot.colorbar()
pyplot.title('Laplacian Filter - Edges Detected')
pyplot.tight_layout()
pyplot.show()

In [ ]:
timeit_cython = %timeit -o laplacian_filter_py(mat_flat, m, n, mat_edges_flat)
time_cython = timeit_cython.best

# save the value into a dictionary
time_results['Cython'] = time_cython

pyplot.bar(time_results.keys(), time_results.values())
pyplot.ylabel('Time (s)')
pyplot.title('Execution Time')
pyplot.xticks(rotation=45)
pyplot.tight_layout()
pyplot.yscale('log')  # set y-axis to log scale
pyplot.show()

# Ctypes
https://docs.python.org/3/library/ctypes.html

In [ ]:
%%file laplacian_filter.c

#include <stdio.h>
#include <stdlib.h>

void laplacian_filter(double *mat, const int m, const int n, double *retval)
{
    for( int r = 1; r < m - 1; r++)
    {

        const int offset = r*n;

        for( int c = 1; c < n - 1; c++)
        {
            const int i = offset + c;
            const int i_top = offset + n + c;
            const int i_bottom = offset - n + c;
            const int i_left = offset + (c-1);
            const int i_right = offset + (c+1);

            retval[i] = -4.0*mat[i] + mat[i_bottom] + mat[i_top] + mat[i_left] + mat[i_right];
        }
    }
}

## Using gcc

In [ ]:
!gcc laplacian_filter.c -std=c99 -shared -fPIC -O3 -mavx2 -o laplacian_filter_gcc.so

In [ ]:
!ls -l laplacian_filter_gcc.so

In [ ]:
import os
import ctypes

# load the function from the shared library
clib = ctypes.cdll.LoadLibrary(os.path.join(os.getcwd(), 'laplacian_filter_gcc.so'))
clib.laplacian_filter.argtypes = [
    ctypes.POINTER(ctypes.c_double),
    ctypes.c_int,
    ctypes.c_int,
    ctypes.POINTER(ctypes.c_double),
]


def laplacian_filter_ctypes_gcc(mat, m, n, retval):
    mat_ptr = mat.ctypes.data_as(ctypes.POINTER(ctypes.c_double))
    retval_ptr = retval.ctypes.data_as(ctypes.POINTER(ctypes.c_double))

    clib.laplacian_filter(mat_ptr, m, n, retval_ptr)

In [ ]:
laplacian_filter_ctypes_gcc(mat_flat, m, n, mat_edges_flat)
pyplot.imshow(mat_edges_flat.reshape(m, n) > 0, cmap='gray')
pyplot.colorbar()
pyplot.title('Laplacian Filter - Edges Detected')
pyplot.tight_layout()
pyplot.show()

In [ ]:
timeit_ctypes = %timeit -o laplacian_filter_ctypes_gcc(mat_flat, m, n, mat_edges_flat)
time_ctypes = timeit_ctypes.best

# save the value into a dictionary
time_results['Ctypes GCC'] = time_ctypes

pyplot.bar(time_results.keys(), time_results.values())

# find the minimum time and plot it in red
min_time = min(time_results.values())
min_time_key = [key for key in time_results if time_results[key] == min_time][0]

# for each bar plot the inverse of the speedup of the minimum time
# format the text to show these as integrers inversed speedup is greater than 100
for key, value in time_results.items():
    inv_speedup = 1/min_time*value
    pyplot.text(key, value, f'{int(inv_speedup)}x' if inv_speedup > 10 else f'{inv_speedup:.2f}x', ha='center')
pyplot.grid(axis='y')
pyplot.bar(min_time_key, min_time, color='red')
pyplot.ylabel('Time (s)')
pyplot.title('Execution Time')
pyplot.xticks(rotation=90)
pyplot.tight_layout()
pyplot.yscale('log')  # set y-axis to log scale
pyplot.show()

## Using the Intel compiler (icc) [if you have it]

In [ ]:
# !icc laplacian_filter.c -std=gnu99 -shared -fPIC -Ofast -O3 -fno-strict-aliasing -limf -o laplacian_filter_icc.so
!icc laplacian_filter.c -std=gnu99 -shared -fPIC -march=core-avx2 -limf -o laplacian_filter_icc.so

In [ ]:
!ls -l laplacian_filter_icc.so

In [ ]:
import ctypes

# load the function from the shared library
clib = ctypes.cdll.LoadLibrary(os.path.join(os.getcwd(), 'laplacian_filter_icc.so'))
clib.laplacian_filter.argtypes = [
    ctypes.POINTER(ctypes.c_double),
    ctypes.c_int,
    ctypes.c_int,
    ctypes.POINTER(ctypes.c_double),
]


def laplacian_filter_ctypes_icc(mat, m, n, retval):
    mat_ptr = mat.ctypes.data_as(ctypes.POINTER(ctypes.c_double))
    retval_ptr = retval.ctypes.data_as(ctypes.POINTER(ctypes.c_double))

    clib.laplacian_filter(mat_ptr, m, n, retval_ptr)

m, n = mat.shape
mat_flat = mat.flatten()
mat_edges_flat = numpy.zeros_like(mat_flat)

laplacian_filter_ctypes_icc(mat_flat, m, n, mat_edges_flat)
pyplot.imshow(mat_edges_flat.reshape(m, n) > 0, cmap='gray')

In [ ]:
%timeit laplacian_filter_ctypes_icc(mat_flat, m, n, mat_edges_flat)

# comparison between ctypes + gcc, ctypes + icc and numba

.. todo:: no comparison yet

## Moving forward - Large array that do not fit in the CPU cash

In [ ]:
mat_large = imread('sample_large.jpg').astype('f8')
pyplot.imshow(mat_large, cmap='gray')

m_large, n_large = mat_large.shape
mat_large_flat = mat_large.flatten()
mat_large_edges_flat = numpy.zeros_like(mat_large_flat)

In [ ]:
%timeit laplacian_filter_ctypes_gcc(mat_large_flat, m_large, n_large, mat_large_edges_flat)

In [ ]:
%timeit laplacian_filter_ctypes_icc(mat_large_flat, m_large, n_large, mat_large_edges_flat)

In [ ]:
mat_large_edges = mat_edges = numpy.zeros_like(mat_large)
%timeit laplacian_filter_numba_row(mat_large, mat_edges)